## 1. Experimental Setup
1.1 Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1.2 Bioconda setup

In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 76 (delta 26), reused 59 (delta 15), pack-reused 0
Receiving objects: 100% (76/76), 318.16 KiB | 10.97 MiB/s, done.
Resolving deltas: 100% (26/26), done.
./
./root/
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.profile
./root/.bashrc.biolab
./root/.condarc
./root/.tmux.conf
./root/.vimrc
--2024-06-06 06:02:07--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143808873 (137M) [application/octet-stream]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>] 137.15M  

1.3 Install *samtools* and *biopython* for analysis

In [3]:
!conda install -y bedtools bioawk samtools
!pip install biopython

Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: | / - done

## Package Plan ##

  environment location: /root/conda

  added / updated specs:
    - bedtools
    - bioawk
    - samtools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    bedtools-2.31.1            |       hf5e1c6e_1         1.5 MB  bioconda
    bioawk-1.0                 |      he4a0461_10         198 KB  bioconda
    ca-certificates-2024.6.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    conda-24.5.0               |  py312h7900ff3_0         1.2 MB  conda-forge
    frozendict-2.4.4           |  py312h9a8786e_0          30 KB  conda-forge
    

1.4 Acess the given sequencing data


In [ ]:
!mkdir -p /content/drive/MyDrive/binfo1-work
%cd /content/drive/MyDrive/binfo1-work
!cp ../binfo1-datapack1/* .

In [4]:
%cd /content/drive/MyDrive/binfo1-work/

/content/drive/MyDrive/binfo1-work


## 2. Find the Lin28a binding site
2.1 Search the coordinates of the gene whose "transcript_support_level" is 1 (chr4, 133730641-133746152)

In [ ]:
!grep -i lin28a ../binfo1-datapack1/gencode.gtf

chr9	ENSEMBL	gene	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; gene_type "miRNA"; gene_name "Mirlet7g"; level 3; mgi_id "MGI:2676800";
chr9	ENSEMBL	transcript	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";
chr9	ENSEMBL	exon	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; exon_number 1; exon_id "ENSMUSE00000522665.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";
^C


2.2 Extract a samfile where reads are located around the coordinates

In [ ]:
!samtools view -b -o Lin28a.bam ../binfo1-datapack1/CLIP-35L33G.bam chr4:133730641-133746152
!samtools view Lin28a.bam | wc -l

4293


In [ ]:
!samtools mpileup Lin28a.bam > Lin28a.pileup
!wc -l Lin28a.pileup

[mpileup] 1 samples in 1 input files
144525 Lin28a.pileup


In [ ]:
!awk '$2 >= 133730641 && $2 <= 133746152 { print $0; }' Lin28a.pileup > Lin28a-gene.pileup
!head Lin28a-gene.pileup

chr4	133730641	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730642	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730643	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730644	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730645	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730646	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730647	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730648	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730649	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730650	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB


2.3 Remove sequencing tags

In [5]:
import pandas as pd
import re

pileup = pd.read_csv('Lin28a-gene.pileup', sep='\t', names=['chr', 'pos', '_ref', 'count', 'basereads', 'quals'])
toremove = re.compile('[<>$*#^]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

pileup[['chr', 'pos', 'matches']]

,chr,pos,matches
0,chr4,133730641,
1,chr4,133730642,
2,chr4,133730643,
3,chr4,133730644,
4,chr4,133730645,
...,...,...,...
15507,chr4,133746148,
15508,chr4,133746149,
15509,chr4,133746150,
15510,chr4,133746151,


2.4 Calculate error frequency of each base position using Shannon's Entropy (minus strand errors are reverse-complemented)



In [6]:
import math

countDict = {}
rc = {'A':'t', 'C':'g', 'G':'c', 'T':'a'}
for pos in pileup['pos']:
  baseCount = {'A':0, 'C':0, 'G':0, 'T':0}
  for base in baseCount:
    baseCount[base] = pileup[pileup['pos'] == pos].iloc[0]['matches'].count(base)
  mbaseCount = {'a':0, 'c':0, 'g':0, 't':0}
  for base in mbaseCount:
    mbaseCount[base] = pileup[pileup['pos'] == pos].iloc[0]['matches'].count(base)
  for base in baseCount:
    baseCount[base] += mbaseCount[rc[base]]
  countDict[pos] = baseCount

In [7]:
entropyDict = {}
for pos in countDict:
  total = sum(list(countDict[pos].values()))
  entropy = -sum((count / total) * math.log2(count / total) for count in list(countDict[pos].values()) if count > 0)
  entropyDict[pos] = entropy

2.5 Filter "Entropy > 0.8" & "read depth > 50" to find high-error rate areas, which can be a candidate of most-frequently cross-linked site

In [8]:
POS = []
for pos in countDict:
  if sum(list(countDict[pos].values())) > 50 and entropyDict[pos] > 0.8:
    POS.append(pos)

for pos in POS:
  print(pos, max(countDict[pos], key=countDict[pos].get))

133730678 G
133730901 T
133730915 G
133731610 G
133731849 G
133731920 G
133732431 G
133732901 A
133732964 G
133733184 A
133735260 G
133735329 G
133735370 G
133735387 G


2.6 Find the binding motif corresponding to the high-error rate areas from the UCSC reference genome

In [ ]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/mm39/chromosomes/chr4.fa.gz
!gunzip chr4.fa.gz

--2024-05-31 02:01:36--  https://hgdownload.soe.ucsc.edu/goldenPath/mm39/chromosomes/chr4.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50063085 (48M) [application/x-gzip]
Saving to: ‘chr4.fa.gz’

chr4.fa.gz          100%[===================>]  47.74M  40.8MB/s    in 1.2s    

2024-05-31 02:01:37 (40.8 MB/s) - ‘chr4.fa.gz’ saved [50063085/50063085]



In [ ]:
from Bio import SeqIO
refSeq = SeqIO.read('chr4.fa',"fasta")

motifs = []
for pos in POS:
  motif_pos = [pos-2,pos-1,pos,pos+1,pos+2,pos+3]
  motif = ''.join(refSeq[pos2] for pos2 in motif_pos)
  motifs.append(motif)

['TCCCAG',
 'AAATTC',
 'TCACAA',
 'TCTCAA',
 'CCGTTT',
 'CCTTTA',
 'TCAATC',
 'CTCCCC',
 'ACCAAA',
 'CTCCCC',
 'CTTTTG',
 'CCTTGG',
 'TCCTTG',
 'CCCTTC']

In [ ]:
import pandas as pd
import re

pileup = pd.read_csv('CLIP-let7g-gene.pileup', sep='\t', names=['chr', 'pos', '_ref', 'count', 'basereads', 'quals'])
toremove = re.compile('[<>$*#^]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

import math

countDict = {}
rc = {'A':'t', 'C':'g', 'G':'c', 'T':'a'}
for pos in pileup['pos']:
  baseCount = {'A':0, 'C':0, 'G':0, 'T':0}
  for base in baseCount:
    baseCount[base] = pileup[pileup['pos'] == pos].iloc[0]['matches'].count(base)
  mbaseCount = {'a':0, 'c':0, 'g':0, 't':0}
  for base in mbaseCount:
    mbaseCount[base] = pileup[pileup['pos'] == pos].iloc[0]['matches'].count(base)
  for base in baseCount:
    baseCount[base] += mbaseCount[rc[base]]
  countDict[pos] = baseCount

entropyDict = {}
for pos in countDict:
  total = sum(list(countDict[pos].values()))
  entropy = -sum((count / total) * math.log2(count / total) for count in list(countDict[pos].values()) if count > 0)
  entropyDict[pos] = entropy

POS = []
for pos in countDict:
  if sum(list(countDict[pos].values())) > 50 and entropyDict[pos] > 0.4:
    POS.append(pos)

!wget https://hgdownload.soe.ucsc.edu/goldenPath/mm39/chromosomes/chr9.fa.gz
!gunzip chr9.fa.gz

from Bio import SeqIO
refSeq = SeqIO.read('chr9.fa',"fasta")

motifs = []
for pos in POS:
  motif_pos = [pos-2,pos-1,pos,pos+1,pos+2,pos+3]
  motif = ''.join(refSeq[pos2] for pos2 in motif_pos)
  motifs.append(motif)

motifs

--2024-05-31 03:55:01--  https://hgdownload.soe.ucsc.edu/goldenPath/mm39/chromosomes/chr9.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39823077 (38M) [application/x-gzip]
Saving to: ‘chr9.fa.gz’

chr9.fa.gz          100%[===================>]  37.98M  36.2MB/s    in 1.0s    

2024-05-31 03:55:02 (36.2 MB/s) - ‘chr9.fa.gz’ saved [39823077/39823077]



['TACAGG', 'CAGGAG', 'AGGAGA']

2.7 Check whether the Lin28A binding motif, "AAGNHG", is contained in the motif candidates

*H: A, C, or U

2.1 Generate a pileup file to find high error-rate areas at a whole genome-wide level

In [ ]:
#!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup
!gunzip CLIP-35L33G.pileup.gz

Following preprocessing & analyzing is going to be done by R using my lab server due to the MemoryError

## 3. Visualize